# 4: Caption postprocessing
In this notebook, we replace 'group' by the subject, for example
A 'group' label with a caption 'a group of men in the street' is changed into 'men'

In [2]:
import pickle
import sys
from pathlib import Path

import pandas as pd

import src.GLIP.maskrcnn_benchmark as maskrcnn_benchmark

sys.modules['maskrcnn_benchmark'] = maskrcnn_benchmark


In [11]:
phrase_grounding_selected_folder = '../data/phrase_grounding_selected/'
dataset_for_phrase_grounding = '../data/dataset_for_phrase_grounding/'


In [6]:
# iterate over files in
# that directory
files = Path(phrase_grounding_selected_folder).glob('*.p')

pickle_filename = []
for file in files:
    pickle_filename.append(str(file))
pickle_filename

['../data/phrase_grounding_selected/JOMU_32980_2k_324w.p',
 '../data/phrase_grounding_selected/CAPO_02480_2k_324w.p',
 '../data/phrase_grounding_selected/JATH_26232_2k_324w.p',
 '../data/phrase_grounding_selected/JATH_10616_2k_324w.p',
 '../data/phrase_grounding_selected/HAWI_01023_2k_324w.p',
 '../data/phrase_grounding_selected/ALRA_000208_2k_324w.p',
 '../data/phrase_grounding_selected/CAPO_00098_2k_324w.p',
 '../data/phrase_grounding_selected/MOAH_04404_2k_324w.p',
 '../data/phrase_grounding_selected/PRMA_00043_2k_324w.p',
 '../data/phrase_grounding_selected/BERA_02799_2k_324w.p',
 '../data/phrase_grounding_selected/HAWI_00863_2k_324w.p',
 '../data/phrase_grounding_selected/JOMU_32035_2k_324w.p',
 '../data/phrase_grounding_selected/PRMA_00200_2k_324w.p',
 '../data/phrase_grounding_selected/PRMA_00503_2k_324w.p',
 '../data/phrase_grounding_selected/CAPO_01180_2k_324w.p',
 '../data/phrase_grounding_selected/JATH_26363_2k_324w.p',
 '../data/phrase_grounding_selected/PRFR_06230_2k_324w.

In [7]:
dataset = pd.DataFrame()
for pickle_file in pickle_filename:
    dataset = dataset.append(pickle.load(open(pickle_file, 'rb')), ignore_index=True)

/tmp/ipykernel_30039/3005811377.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(pickle.load(open(pickle_file, 'rb')), ignore_index=True)
/tmp/ipykernel_30039/3005811377.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(pickle.load(open(pickle_file, 'rb')), ignore_index=True)
/tmp/ipykernel_30039/3005811377.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(pickle.load(open(pickle_file, 'rb')), ignore_index=True)
/tmp/ipykernel_30039/3005811377.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(pickle.load(open(pickle_file, 'rb')),

In [9]:
dataset

,idx,filename,caption,results,model,expr
0,1027,JOMU_32980_2k_324w.jpg,people buying sweets at the market.,"[[tensor(0.8280), tensor(0.7924), tensor(0.689...",GLIP,caption
1,183,CAPO_02480_2k_324w.jpg,a group of soldiers stand in front of a building.,"[[tensor(0.8242), tensor(0.5884), tensor(0.578...",GLIP,caption
2,543,JATH_26232_2k_324w.jpg,soldiers stand in front of a military vehicle.,"[[tensor(0.7898), tensor(0.7305), tensor(0.687...",GLIP,caption
3,430,JATH_10616_2k_324w.jpg,women walking down a street.,"[[tensor(0.6900), tensor(0.6777), tensor(0.667...",GLIP,caption
4,408,HAWI_01023_2k_324w.jpg,a photograph of a large tropical cyclone.,"[[tensor(0.7263)], [[tensor(4.5861), tensor(13...",GLIP,caption
...,...,...,...,...,...,...
1446,1415,PRMA_00341_2k_324w.jpg,monarch and noble person celebrate the wedding...,"[[tensor(0.6181), tensor(0.6012), tensor(0.543...",GLIP,caption
1447,1028,JOMU_33020_2k_324w.jpg,a group of men lighting a fire in the dark.,"[[tensor(0.6246), tensor(0.6187)], [[tensor(11...",GLIP,caption
1448,314,HAWI_00121_2k_324w.jpg,Woman in a boat on Lake Morat,"[[tensor(0.8233), tensor(0.6113)], [[tensor(12...",GLIP,title
1449,343,HAWI_00390_2k_324w.jpg,fishing boat on the beach.,"[[tensor(0.7044)], [[tensor(1.4082), tensor(15...",GLIP,caption


In [10]:
dataset['conf'] = dataset.results.apply(lambda x: x[0])
dataset['bbox'] = dataset.results.apply(lambda x: x[1])
dataset['labels'] = dataset.results.apply(lambda x: x[2])
dataset = dataset.drop(columns='results')

In [12]:
expr_dataset = pickle.load(open(dataset_for_phrase_grounding + 'dataset.p', 'rb'))
expr_dataset = pd.DataFrame(expr_dataset)
expr_dataset['caption_raw'] = expr_dataset.caption.apply(lambda x: x['raw'])
expr_dataset['caption_preprocessed'] = expr_dataset.caption.apply(lambda x: x['preprocessed'])
expr_dataset['title_raw'] = expr_dataset.title.apply(lambda x: x['raw'])
expr_dataset['title_preprocessed'] = expr_dataset.title.apply(lambda x: x['preprocessed'])
expr_dataset = expr_dataset.drop(columns=['caption', 'title'])
expr_dataset.head()

,image_id,filename,caption_raw,caption_preprocessed,title_raw,title_preprocessed
0,2,ALCU_00005_2k_324w.jpg,crowds gather to watch the parade.,crowds gather to watch the parade.,Inauguration of the commemorative plaque on th...,inauguration of the commemorative plaque on th...
1,4,ALCU_00033_2k_324w.jpg,painting artist in his studio.,painting artist in his studio.,"Patient of Dr Xavier Cuony, city physicist, Fr...","patient of dr xavier cuony, city physicist,"
2,5,ALNU_00015_2k_324w.jpg,portrait of a man sitting on a bench.,a man sitting on a bench.,"Elderly (Aloïs Catillaz), Felsenegg, Wünnewil","elderly (aloïs catillaz), felsenegg, wünnewil"
3,6,ALNU_00016_2k_324w.jpg,portrait of a man with a bicycle.,a man with a bicycle.,Male (Hermann Nussbaumer) with his bike,male (hermann nussbaumer) with his bike
4,7,ALNU_00052_2k_324w.jpg,women standing in front of a church.,women standing in front of a church.,"Two women, one in regional costume","two women, one in regional costume"


In [13]:
dataset = pd.merge(dataset, expr_dataset, left_on='filename', right_on='filename', how='left')

In [14]:
dataset

,idx,filename,caption,model,expr,conf,bbox,labels,image_id,caption_raw,caption_preprocessed,title_raw,title_preprocessed
0,1027,JOMU_32980_2k_324w.jpg,people buying sweets at the market.,GLIP,caption,"[tensor(0.8280), tensor(0.7924), tensor(0.6899...","[[tensor(178.8594), tensor(116.5122), tensor(2...","[people, people, people, people, people, peopl...",1488.0,people buying sweets at the market.,people buying sweets at the market.,"Saint Nicholas Festival Market, Place de Notre...","saint nicholas festival market, place de notre..."
1,183,CAPO_02480_2k_324w.jpg,a group of soldiers stand in front of a building.,GLIP,caption,"[tensor(0.8242), tensor(0.5884), tensor(0.5781...","[[tensor(0.8775), tensor(131.9302), tensor(81....","[soldiers, a building, a building, a building]",516.0,a group of soldiers stand in front of a building.,a group of soldiers stand in front of a building.,[Mobilization at Perolles in August 1914],[mobilization at perolles in august 1914]
2,543,JATH_26232_2k_324w.jpg,soldiers stand in front of a military vehicle.,GLIP,caption,"[tensor(0.7898), tensor(0.7305), tensor(0.6872...","[[tensor(97.0861), tensor(137.5495), tensor(14...","[soldiers, soldiers, a military vehicle, soldi...",950.0,soldiers stand in front of a military vehicle.,soldiers stand in front of a military vehicle.,"Additional service for women, Barracks de la P...","additional service for women, barracks de la p..."
3,430,JATH_10616_2k_324w.jpg,women walking down a street.,GLIP,caption,"[tensor(0.6900), tensor(0.6777), tensor(0.6676...","[[tensor(125.4207), tensor(111.6184), tensor(1...","[women, women, women, a street, women]",836.0,women walking down a street.,women walking down a street.,Procession on the route to the Alps during a w...,procession on the route to the alps during a w...
4,408,HAWI_01023_2k_324w.jpg,a photograph of a large tropical cyclone.,GLIP,caption,[tensor(0.7263)],"[[tensor(4.5861), tensor(13.1419), tensor(318....",[a large tropical cyclone],814.0,a photograph of a large tropical cyclone.,a a large tropical cyclone.,Tornado over Lake Morat,tornado over lake morat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1415,PRMA_00341_2k_324w.jpg,monarch and noble person celebrate the wedding...,GLIP,caption,"[tensor(0.6181), tensor(0.6012), tensor(0.5435)]","[[tensor(123.6364), tensor(112.6870), tensor(1...","[noble person, businessperson, monarch]",2100.0,monarch and noble person celebrate the wedding...,monarch and noble person celebrate the wedding...,"Celebration of the Fête-Dieu, the rest of the ...","celebration of the fête-dieu, the rest of the ..."
1450,1028,JOMU_33020_2k_324w.jpg,a group of men lighting a fire in the dark.,GLIP,caption,"[tensor(0.6246), tensor(0.6187)]","[[tensor(110.1859), tensor(209.3674), tensor(1...","[a fire, a group]",1489.0,a group of men lighting a fire in the dark.,a group of men lighting a fire in the dark.,"Cortège de la Saint-Nicolas, torchbearers, Fri...","cortège de la saint-nicolas, torchbearers,"
1451,314,HAWI_00121_2k_324w.jpg,Woman in a boat on Lake Morat,GLIP,title,"[tensor(0.8233), tensor(0.6113)]","[[tensor(123.4919), tensor(130.4832), tensor(2...","[woman, lake morat]",705.0,portrait of a young woman.,a young woman.,Woman in a boat on Lake Morat,woman in a boat on lake morat
1452,343,HAWI_00390_2k_324w.jpg,fishing boat on the beach.,GLIP,caption,[tensor(0.7044)],"[[tensor(1.4082), tensor(159.4477), tensor(136...",[boat],736.0,fishing boat on the beach.,fishing boat on the beach.,"Waves breaking on the jetty and rowing boat, L...","waves breaking on the jetty and rowing boat, l..."


In [15]:
def remove_group(elem):
    if ('a group' in elem['labels']) or ('group' in elem['labels']):
        if elem['expr'] == 'caption':
            expression = elem['caption_preprocessed']
        else:
            expression = elem['title_preprocessed']
        expression = expression.split(' ')
        try:
            word = expression[expression.index('of') + 1]
            if word == 'young':
                word = word + ' ' + expression[expression.index('of') + 2]
            return [word if label == 'a group' else label for label in elem['labels']]
        except:
            return elem['labels']

    else:
        return elem['labels']

In [16]:
dataset['labels'] = dataset.apply(lambda x: remove_group(x), axis=1)

In [17]:
pickle.dump(dataset, open('../data/dataset_for_segmentation.p','wb'))